In [ ]:
!pip install langchain langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
# create tool

@tool
def get_currency_conversion_factor(base_currency:str,target_currency:str)->float:
  """
  This fucntion will fetch the currency conversion factor between base_currency and target_currency
  """
  url=f'https://v6.exchangerate-api.com/v6/your_key/pair/{base_currency}/{target_currency}'
  response=requests.get(url)

  return response.json()



In [ ]:
get_currency_conversion_factor.invoke({'base_currency':"USD",'target_currency':"EUR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1771027201,
 'time_last_update_utc': 'Sat, 14 Feb 2026 00:00:01 +0000',
 'time_next_update_unix': 1771113601,
 'time_next_update_utc': 'Sun, 15 Feb 2026 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'EUR',
 'conversion_rate': 0.8427}

In [ ]:
@tool
def covert(amount:float,conversion_factor:float)->float:
    """
    This function will convert the amount from base currency to target currency
    """
    return amount*conversion_factor

In [ ]:
covert.invoke({'amount':10,'conversion_factor':0.8427})

8.427

In [ ]:
# tool binding
llm = ChatOpenAI()

In [ ]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [ ]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_PKL8v7zwmphzNel0MnvjjGvY',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_vRdld30yHTKFGcTQPumgzH5u',
  'type': 'tool_call'}]

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PKL8v7zwmphzNel0MnvjjGvY', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_vRdld30yHTKFGcTQPumgzH5u', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 123, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BR8rovupYuNnIbGO41TOO3knQ2Cr9', 'finish_reason': 'too

In [ ]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.0117. \n\nBased on this conversion factor, 10 INR is equal to 0.117 USD.'